# **ANALISIS FINANCIERO EMPRESA ACUAMAR S.A**

### **1. EDA Dashboard y Alertas de Liquidez:**

**Indicadores Histórico:** Tarjetas de KPI: Días de Cartera promedio, Capital de Trabajo Neto (actual), Margen Bruto (%).	Muestra la salud financiera actual.

**Tendencia de Caja:** |Gráfico de Línea Dual| Tendencia histórica de Cobros vs. Pagos.	Identifica descalces históricos de caja.

**Alertas de Liquidez:** |Gráfico de Línea del Flujo de Caja Acumulado proyectado, con una banda roja marcando el umbral de Liquidez Mínima ($0 o límite de seguridad del cliente)|.	Visualiza cuándo y en qué magnitud se necesitaría financiación externa (alerta preventiva).


### **2. Flujo de Caja Proyectado y Control de Impuestos.**

Predicción Base: Aplicar el modelo Prophet (Corto/Mediano Plazo) a las series Cobros_Clientes y Pagos_Proveedores por separado.

**a) Cálculo Proyectado:** Sumar los flujos predichos con el Efectivo_Inicial histórico para obtener el Flujo de Caja Neto Proyectado.

**Visualizaciones:**
Flujo de Caja Proyectado |Gráfico de Área Acumulada |Muestra las Entradas (Cobros) y Salidas (Pagos) proyectadas, visualizando el Flujo Neto como la brecha entre ellas.



**b) Cálculo de Impuestos:** Usar las métricas predichas (Ventas_Gravadas, Gasto_Nomina) y aplicar las tasas impositivas colombianas ($19\%$ IVA general, Retención en la Fuente, Tasa de Renta) para obtener los valores a pagar en el futuro.

**Visualizaciones**: Control de Impuestos	Gráfico de Barras Agrupadas: Proyección de las obligaciones de IVA, Retención en la Fuente y Renta para los próximos 12 meses.



### **3. Predicción de Riesgos Financieros (Análisis ARIMA + Monte Carlo)**
Para predecir el Riesgo Financiero (la probabilidad de que el Flujo de Caja sea negativo), se combina la precisión de ARIMA con la simulación de riesgo de Monte Carlo.

**a) Modelado de la Media (ARIMA):**
Se utiliza ARIMA/Prophet para obtener la predicción promedio de Cobros_Clientes y la Desviación Estándar (volatilidad) de la predicción.


**b)	Simulación de Riesgo (Monte Carlo):**

Se ejecutan miles de escenarios de Flujo de Caja futuro. En cada escenario, la métrica Cobros_Clientes se ajusta aleatoriamente usando la media ARIMA y la desviación estándar histórica, simulando la incertidumbre del mercado.


**Visualización de Riesgo:**

**Gráfico de Líneas Múltiples:** Muestra una banda de escenarios (ej. 50 simulaciones) del Flujo de Caja Acumulado, destacando los caminos extremos (optimista vs. pesimista).

**Tarjeta de KPI:** Muestra la Probabilidad de Fallo de Liquidez (el porcentaje de las simulaciones de Monte Carlo donde el Flujo de Caja Acumulado proyectado cae por debajo de cero).

**Histograma de Resultados:** Muestra la distribución de los posibles Saldos Finales de Caja al término del horizonte de predicción.



## **Instalacion de Librerias**

In [1]:
!pip install scikit-learn
!pip install yfinance prophet
!pip install tqdm
!pip install pmdarima
!pip install kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.1/689.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.9 MB/s eta 0:00:00


In [2]:
# 1. INSTALACIONES E IMPORTACIONES
# ==============================================================================

# Instalaciones (Solo es necesario si no están ya instaladas en su entorno)
#!pip install pandas scikit-learn plotly numpy yfinance prophet

# Importaciones
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from datetime import datetime
from datetime import timedelta
from google.colab import drive
import os
import math
from prophet import Prophet
from tqdm.notebook import tqdm
# importar pmdarima
import pmdarima as pm
from pmdarima.model_selection import train_test_split
from pmdarima.arima import auto_arima

print("Librerías instaladas e importadas correctamente.")

Librerías instaladas e importadas correctamente.


## **Lectura y Carga de datos.**

In [3]:
# Lectura de datos excel.
df_fin = pd.read_excel('Datos_Financieros_Comerciales.xlsx', sheet_name='financiero')
print(df_fin)

         FECHA  Efectivo_Inicial  Cobros_Clientes  Pagos_Proveedores  \
0   2016-01-01          59086650        188586016           90204672   
1   2016-02-01          77235004        163313024           88542133   
2   2016-03-01          70549017        133988503           93411250   
3   2016-04-01          62986749        126123821           90732394   
4   2016-05-01          55490160        124633294           89246303   
..         ...               ...              ...                ...   
115 2025-08-01          78099899        254413948          129723532   
116 2025-09-01          76018676        230071589          131801874   
117 2025-10-01          66307137        242307220          129977915   
118 2025-11-01          78968326        257867211          134368709   
119 2025-12-01          69937869        279377655          138868326   

     Ventas_Gravadas  Gastos_Gravados  Base_Retencion  Gasto_Nomina  
0       1.760220e+08     6.008749e+07    1.681215e+08      197500

In [4]:
# Datos Nulos
df_fin.isnull().sum()

,0
FECHA,0
Efectivo_Inicial,0
Cobros_Clientes,0
Pagos_Proveedores,0
Ventas_Gravadas,0
Gastos_Gravados,0
Base_Retencion,0
Gasto_Nomina,0


In [5]:
# Datos duplicados
df_fin.duplicated().sum()

np.int64(0)

In [6]:
# Estadisticas descriptivas
df_fin.describe()

,FECHA,Efectivo_Inicial,Cobros_Clientes,Pagos_Proveedores,Ventas_Gravadas,Gastos_Gravados,Base_Retencion,Gasto_Nomina
count,120,1.200000e+02,1.200000e+02,1.200000e+02,1.200000e+02,1.200000e+02,1.200000e+02,1.200000e+02
mean,2020-12-15 18:00:00,6.448351e+07,1.936847e+08,1.125438e+08,1.785366e+08,6.803502e+07,1.639072e+08,2.753945e+07
min,2016-01-01 00:00:00,4.868567e+07,1.206890e+08,8.854213e+07,1.132227e+08,4.794827e+07,9.986984e+07,1.975007e+07
25%,2018-06-23 12:00:00,5.784240e+07,1.655425e+08,1.015566e+08,1.532832e+08,6.008846e+07,1.410211e+08,2.396152e+07
50%,2020-12-16 12:00:00,6.349641e+07,1.891544e+08,1.133451e+08,1.742154e+08,6.843078e+07,1.578854e+08,2.750325e+07
75%,2023-06-08 12:00:00,7.231194e+07,2.204025e+08,1.232197e+08,2.036883e+08,7.607317e+07,1.856562e+08,3.133917e+07
max,2025-12-01 00:00:00,8.706044e+07,2.808287e+08,1.388683e+08,2.644244e+08,8.966163e+07,2.479645e+08,3.518776e+07
std,NaN,9.288072e+06,3.709316e+07,1.354224e+07,3.445044e+07,1.001352e+07,3.266244e+07,4.418371e+06


In [7]:
# Deteccion de datos atipicos metodo IQR
Q1 = df_fin.quantile(0.25)
Q3 = df_fin.quantile(0.75)

# Calcular IQR
IQR = Q3 - Q1

# Definir limites
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identifica las columnas numericas
print("\n--- Detección de Datos Atípicos (Método IQR) ---")
outliers_found = False
for column in df_fin.select_dtypes(include=np.number).columns:
    col_outliers = df_fin[(df_fin[column] < lower_bound[column]) | (df_fin[column] > upper_bound[column])]
    if not col_outliers.empty:
        outliers_found = True
        print(f"\nOutliers en la columna '{column}':")
        print(col_outliers[['FECHA', column]])

if not outliers_found:
    print("No se encontraron datos atípicos significativos en las columnas numéricas usando el método IQR.")



--- Detección de Datos Atípicos (Método IQR) ---
No se encontraron datos atípicos significativos en las columnas numéricas usando el método IQR.


## **Definir las constantes Financieras**

In [8]:
# ==============================================================================
# 1. SETUP E INICIALIZACIÓN (Librerías y Constantes)
# ==============================================================================

# --- CONSTANTES CODIFICADAS  ---

# [COMENTARIO: Constante: Umbral_Liquidez_Mínima]
UMBRAL_LIQUIDEZ_MINIMA = 0.0      # $0 COP, límite de seguridad para Alertas de Liquidez.

# [COMENTARIO: Constante: Tasa_IVA_General]
TASA_IVA_GENERAL = 0.19           # 19% IVA General en Colombia.

# [COMENTARIO: Constante: Tasa_Impuesto_Renta_Sociedades]
TASA_IMPUESTO_RENTA = 0.35        # 35% Impuesto de Renta para sociedades.

# [COMENTARIO: Constante: Tasa_Retención_Fuente_Estándar]
TASA_RETENCION_FUENTE = 0.025     # 2.5% Tasa de Retención en la Fuente Estándar (sugerida).

# [COMENTARIO: Constante: Tasa_Rotación_Cartera_Días]
TASA_ROTACION_CARTERA_DIAS = 30   # 30 días, estándar para KPI de Días de Cartera.

HORIZONTE_PROYECCION = 12 # Meses a proyectar.
NUM_SIMULACIONES = 5000   # Número de escenarios para Monte Carlo.

print("Librerías importadas y constantes de negocio definidas. ✅")
print(f"\nConstantes definidas:\n- Umbral de Alerta: {UMBRAL_LIQUIDEZ_MINIMA:,.0f} COP\n- Tasa IVA: {TASA_IVA_GENERAL*100}%")

Librerías importadas y constantes de negocio definidas. ✅

Constantes definidas:
- Umbral de Alerta: 0 COP
- Tasa IVA: 19.0%


In [9]:
# Conversión de tipos y limpieza de datos inicial
if not df_fin.empty:
    df_fin['FECHA'] = pd.to_datetime(df_fin['FECHA'], errors='coerce')
    df_fin_mensual = df_fin.dropna(subset=['FECHA']).sort_values('FECHA').set_index('FECHA').resample('M').last()
    df_fin_mensual = df_fin_mensual.dropna(how='all')

/tmp/ipython-input-3983340466.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_fin_mensual = df_fin.dropna(subset=['FECHA']).sort_values('FECHA').set_index('FECHA').resample('M').last()


## **1. EDA Dashboard y Alertas de Liquidez:**

In [10]:
# ==============================================================================
# 2. EDA DASHBOARD Y ALERTAS DE LIQUIDEZ (KPIs Históricos)
# ==============================================================================

print("## \u001f\u001f Indicadores Financieros Históricos")

# ----------------------------------------------------
# CÁLCULO DE KPIS HISTÓRICOS
# ----------------------------------------------------

# 1. Capital de Trabajo Neto (CTN) = (Cobros_Clientes - Pagos_Proveedores) * Días_Pendientes / 360
# Usaremos una aproximación mensual: Activos Ctes (Cobros) - Pasivos Ctes (Pagos)
df_fin['Capital_Trabajo_Neto'] = df_fin['Cobros_Clientes'] - df_fin['Pagos_Proveedores']

# 2. Margen Bruto (%) = ((Ventas_Gravadas - Costo_Venta_Unitario*Volumen_Vendido) / Ventas_Gravadas) * 100
# Dado que 'Costo_Venta_Unitario' y 'Volumen_Vendido' están en el archivo comercial (transaccional),
# se usa una aproximación de Flujo de Caja Bruto / Ventas
df_fin['Margen_Bruto_Aprox'] = (df_fin['Ventas_Gravadas'] - df_fin['Pagos_Proveedores']) / df_fin['Ventas_Gravadas']

# 3. Días de Cartera Promedio (DCP)
# Se necesitan datos a nivel de cliente/transacción, pero para el financiero (mensual),
# se usa una aproximación basada en Ventas: DCP = (Cuentas por Cobrar Promedio / Ventas a Crédito Promedio) * Días del Período
# Usamos la constante TASA_ROTACION_CARTERA_DIAS como referencia.
DIAS_CARTERA_PROMEDIO = df_fin['Ventas_Gravadas'].rolling(window=3).mean().mean() / df_fin['Cobros_Clientes'].mean() * TASA_ROTACION_CARTERA_DIAS


# ----------------------------------------------------
# VISUALIZACIÓN SUGERIDA: Tabla de KPI (Valores más recientes)
# ----------------------------------------------------
print("\n### Tabla de KPI (Valores Recientes)")

kpi_data = [
    {"Indicador": "Días de Cartera Promedio", "Valor Reciente": f"{DIAS_CARTERA_PROMEDIO:.0f} días", "Propósito": f"Salud de cobros (comparar con {TASA_ROTACION_CARTERA_DIAS} días)"},
    {"Indicador": "Capital de Trabajo Neto", "Valor Reciente": f"{df_fin['Capital_Trabajo_Neto'].iloc[-1]:,.0f} COP", "Propósito": "Capacidad de cubrir pasivos a corto plazo"},
    {"Indicador": "Margen Bruto (%) (Aprox)", "Valor Reciente": f"{df_fin['Margen_Bruto_Aprox'].iloc[-1]*100:.2f}%", "Propósito": "Eficiencia en el flujo de caja operativo"}
]
df_kpi = pd.DataFrame(kpi_data)

fig_kpi_table = go.Figure(data=[go.Table(
    header=dict(values=list(df_kpi.columns), fill_color='paleturquoise', align='left'),
    cells=dict(values=[df_kpi['Indicador'], df_kpi['Valor Reciente'], df_kpi['Propósito']], fill_color='lavender', align='left'))
])

fig_kpi_table.update_layout(title_text='Indicadores Financieros Clave (KPIs)')

# Modificaciones solicitadas por el usuario:
fig_kpi_table.update_layout(
    width=600,
    height=400,
    margin=dict(l=40, r=40, t=60, b=40),
    template="plotly_white"
)
fig_kpi_table.show()


##  Indicadores Financieros Históricos

### Tabla de KPI (Valores Recientes)


/usr/local/lib/python3.12/dist-packages/kaleido/_sync_server.py:11: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




In [14]:
# ----------------------------------------------------
# VISUALIZACIÓN SUGERIDA: Gráfico de Línea Dual (Tendencia de Caja)
# ----------------------------------------------------
print("\n### Gráfico de Línea Dual: Tendencia histórica de Cobros vs. Pagos")

# Crear el DataFrame para Plotly
df_plot_caja = df_fin[['FECHA', 'Cobros_Clientes', 'Pagos_Proveedores']]
fig_tendencia_caja = px.line(df_plot_caja, x='FECHA', y=['Cobros_Clientes', 'Pagos_Proveedores'],
                             title='Cobros vs. Pagos: Identificación de Descalces Históricos',
                             labels={'value': 'Monto (COP)', 'FECHA': 'Fecha'},
                             color_discrete_map={'Cobros_Clientes': 'green', 'Pagos_Proveedores': 'red'})
fig_tendencia_caja.update_layout(yaxis_title='Monto (COP)', legend_title='Transacción')
fig_tendencia_caja.update_layout(
    width=900,
    height=450,
    margin=dict(l=40, r=40, t=60, b=40),
    template="plotly_white",
    font=dict(size=12)
)

fig_tendencia_caja.show()


### Gráfico de Línea Dual: Tendencia histórica de Cobros vs. Pagos


## **2. Flujo de Caja Proyectado y Control de Impuestos**

In [15]:
# ==============================================================================
# 3. FLUJO DE CAJA PROYECTADO Y CONTROL DE IMPUESTOS
# ==============================================================================

print("## 📈 Proyección y Control de Flujo de Caja e Impuestos")

def fit_and_predict_prophet(series, horizon):
    """Aplica Prophet a una serie de tiempo y devuelve las predicciones."""
    df_prophet = series.reset_index().rename(columns={'FECHA': 'ds', series.name: 'y'})

    # Prophet se ajusta bien a datos mensuales, aunque se beneficia de estacionalidad (ej: anual)
    m = Prophet(daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=True)
    m.fit(df_prophet)

    future = m.make_future_dataframe(periods=horizon, freq='M')
    forecast = m.predict(future)

    # Solo necesitamos las predicciones futuras
    forecast_future = forecast[forecast['ds'] > series.index.max()].set_index('ds')

    # Devolvemos la predicción (yhat) y la desviación estándar de la predicción (yhat_upper - yhat_lower) / 2
    # La desviación estándar será utilizada en la simulación de Monte Carlo.
    std_dev = (forecast_future['yhat_upper'] - forecast_future['yhat_lower']) / 2

    return forecast_future['yhat'], std_dev.rename('std_dev')



## 📈 Proyección y Control de Flujo de Caja e Impuestos


In [16]:
# ----------------------------------------------------
# PREDICCIÓN BASE (Prophet)
# ----------------------------------------------------
horizonte = HORIZONTE_PROYECCION # 12 meses

print(f"Iniciando predicción a {horizonte} meses para Cobros y Pagos...")
pred_cobros, std_cobros = fit_and_predict_prophet(df_fin_mensual['Cobros_Clientes'], horizonte)
pred_pagos, _ = fit_and_predict_prophet(df_fin_mensual['Pagos_Proveedores'], horizonte)
pred_ventas_gravadas, _ = fit_and_predict_prophet(df_fin_mensual['Ventas_Gravadas'], horizonte)
pred_gasto_nomina, _ = fit_and_predict_prophet(df_fin_mensual['Gasto_Nomina'], horizonte)
pred_base_retencion, _ = fit_and_predict_prophet(df_fin_mensual['Base_Retencion'], horizonte)

Iniciando predicción a 12 meses para Cobros y Pagos...


/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [17]:
# ----------------------------------------------------
# CÁLCULO PROYECTADO: Flujo de Caja Neto
# ----------------------------------------------------

# Creación del DataFrame de Proyección
df_proj = pd.DataFrame({
    'Cobros_Proyectados': pred_cobros.values,
    'Pagos_Proyectados': pred_pagos.values,
}, index=pred_cobros.index)

df_proj['Flujo_Neto_Proyectado'] = df_proj['Cobros_Proyectados'] - df_proj['Pagos_Proyectados']

# Efectivo Inicial del período de proyección (último valor histórico)
efectivo_inicial_actual = df_fin_mensual['Efectivo_Inicial'].iloc[-1]
df_proj['Flujo_Acumulado'] = (df_proj['Flujo_Neto_Proyectado'].cumsum() + efectivo_inicial_actual)


In [18]:
# ----------------------------------------------------
# CÁLCULO DE IMPUESTOS (Obligaciones Fiscales Proyectadas)
# ----------------------------------------------------
# 1. Provisión IVA (a pagar/favor): (Ventas_Gravadas * TASA_IVA_GENERAL) - (Gastos_Gravados * TASA_IVA_GENERAL)
# Usaremos una aproximación basada en Ventas - Gastos Fiscales.
pred_gastos_gravados, _ = fit_and_predict_prophet(df_fin_mensual['Gastos_Gravados'], horizonte)
df_proj['Proyeccion_IVA'] = (pred_ventas_gravadas - pred_gastos_gravados) * TASA_IVA_GENERAL

# 2. Provisión Retención en la Fuente: Base_Retencion * TASA_RETENCION_FUENTE
df_proj['Proyeccion_Retencion_Fuente'] = pred_base_retencion * TASA_RETENCION_FUENTE

# 3. Provisión Impuesto de Renta (mensual): (Base Gravable Estimada / 12) * TASA_IMPUESTO_RENTA
# Para simplificar, asumimos que la Utilidad Bruta del período es base para provisión mensual.
df_proj['Proyeccion_Impuesto_Renta'] = (df_proj['Flujo_Neto_Proyectado'] * TASA_IMPUESTO_RENTA) / 12
df_proj['Obligaciones_Fiscales_Totales'] = df_proj['Proyeccion_IVA'] + df_proj['Proyeccion_Retencion_Fuente'] + df_proj['Proyeccion_Impuesto_Renta']


/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [19]:
# ----------------------------------------------------
# VISUALIZACIÓN SUGERIDA: Gráfico de Área Acumulada (Flujo de Caja Proyectado)
# ----------------------------------------------------
print("\n### Gráfico de Área Acumulada: Entradas vs. Salidas Proyectadas")

fig_flujo_area = go.Figure()
fig_flujo_area.add_trace(go.Scatter(x=df_proj.index, y=df_proj['Cobros_Proyectados'].cumsum(),
                                    fill='tozeroy', mode='lines', name='Entradas Acumuladas (Cobros)',
                                    line_color='green'))
fig_flujo_area.add_trace(go.Scatter(x=df_proj.index, y=df_proj['Pagos_Proyectados'].cumsum(),
                                    fill='tozeroy', mode='lines', name='Salidas Acumuladas (Pagos)',
                                    line_color='red'))
fig_flujo_area.update_layout(title=f'Flujo de Caja Proyectado (Entradas vs. Salidas a {horizonte} meses)',
                              xaxis_title='Fecha de Proyección',
                              yaxis_title='Monto Acumulado (COP)')
fig_flujo_area.update_layout(
    width=900,
    height=450,
    margin=dict(l=40, r=40, t=60, b=40),
    template="plotly_white",
    font=dict(size=12)
)

fig_flujo_area.show()



### Gráfico de Área Acumulada: Entradas vs. Salidas Proyectadas


In [22]:
# ----------------------------------------------------
# VISUALIZACIÓN SUGERIDA: Gráfico de Alertas de Liquidez
# ----------------------------------------------------
print("\n### Alertas de Liquidez: Flujo de Caja Acumulado (Proyección Base)")

fig_alerta = px.line(df_proj.reset_index(), x='ds', y='Flujo_Acumulado',
                    title='Alerta de Liquidez: Flujo de Caja Acumulado Proyectado',
                    labels={'ds': 'Fecha', 'Flujo_Acumulado': 'Flujo Neto Acumulado (COP)'})

# Banda Roja de Umbral de Liquidez
fig_alerta.add_hline(y=UMBRAL_LIQUIDEZ_MINIMA, line_dash="dash", line_color="red",
                     annotation_text=f"Umbral de Liquidez Mínima ({UMBRAL_LIQUIDEZ_MINIMA:,.0f} COP)",
                     annotation_position="top right")

# Marcar puntos donde se cruza el umbral
puntos_alerta = df_proj[df_proj['Flujo_Acumulado'] < UMBRAL_LIQUIDEZ_MINIMA].index
if not puntos_alerta.empty:
    fig_alerta.add_trace(go.Scatter(x=puntos_alerta, y=df_proj.loc[puntos_alerta, 'Flujo_Acumulado'],
                                    mode='markers', name='Riesgo de Liquidez',
                                    marker=dict(color='red', size=10)))

fig_alerta.update_layout(
    width=900,
    height=450,
    margin=dict(l=40, r=40, t=60, b=40),
    template="plotly_white",
    font=dict(size=12)
)

fig_alerta.show()


### Alertas de Liquidez: Flujo de Caja Acumulado (Proyección Base)


In [23]:
# ----------------------------------------------------
# VISUALIZACIÓN SUGERIDA: Control de Impuestos
# ----------------------------------------------------
print("\n### Gráfico de Barras Agrupadas: Proyección de Obligaciones Fiscales")

df_impuestos_plot = df_proj[['Proyeccion_IVA', 'Proyeccion_Retencion_Fuente', 'Proyeccion_Impuesto_Renta']].reset_index().rename(columns={'ds': 'Fecha'})

fig_impuestos = px.bar(df_impuestos_plot, x='Fecha',
                       y=['Proyeccion_IVA', 'Proyeccion_Retencion_Fuente', 'Proyeccion_Impuesto_Renta'],
                       title='Proyección de Obligaciones Fiscales (IVA, Retención, Renta)',
                       labels={'value': 'Monto Proyectado (COP)', 'variable': 'Tipo de Impuesto'})
fig_impuestos.update_layout(xaxis_tickangle=-45)
fig_impuestos.update_layout(
    width=900,
    height=450,
    margin=dict(l=40, r=40, t=60, b=40),
    template="plotly_white",
    font=dict(size=12)
)
fig_impuestos.show()

# Exportar datos para la siguiente celda
global df_proj_impuestos
df_proj_impuestos = df_proj.copy()
global std_cobros_global
std_cobros_global = std_cobros.copy()


### Gráfico de Barras Agrupadas: Proyección de Obligaciones Fiscales


## **3. Predicción de Riesgos Financieros (Monte Carlo)**

In [24]:
# ==============================================================================
# 4. PREDICCIÓN DE RIESGOS FINANCIEROS (Monte Carlo)
# ==============================================================================

print("## 🎲 Simulación de Riesgo Financiero (Monte Carlo)")

# Parámetros de la simulación
num_sims = NUM_SIMULACIONES
horizonte = HORIZONTE_PROYECCION
efectivo_inicial = df_fin['Efectivo_Inicial'].iloc[-1]
# Volatilidad: Usamos la desviación estándar de la predicción de Prophet (std_cobros)
# Para el ejemplo, usamos el promedio de la std dev de los cobros proyectados como sigma
sigma_cobros = std_cobros_global.mean()

# Usaremos los pagos proyectados como el valor esperado (media)
pagos_proyectados_media = df_proj_impuestos['Pagos_Proyectados'].values

# Almacenará los resultados acumulados de cada simulación
resultados_acumulados = []

## 🎲 Simulación de Riesgo Financiero (Monte Carlo)


In [25]:
# ==============================================================================
# BUCLE DE MONTE CARLO (miles de escenarios)
# ==============================================================================
print(f"Ejecutando {num_sims} simulaciones a {horizonte} meses...")

for i in tqdm(range(num_sims)):
    # 1. Simular Cobros: Usar la predicción base (media Prophet) y añadir ruido normal
    # El ruido se simula usando N(0, sigma_cobros)
    cobros_simulados = pred_cobros.values + np.random.normal(0, sigma_cobros, size=horizonte)

    # Asegurar que no haya cobros negativos
    cobros_simulados[cobros_simulados < 0] = 0

    # 2. Calcular Flujo Neto Simulado
    flujo_neto_simulado = cobros_simulados - pagos_proyectados_media

    # 3. Flujo Acumulado Simulado
    flujo_acumulado_simulado = np.cumsum(flujo_neto_simulado) + efectivo_inicial

    resultados_acumulados.append(flujo_acumulado_simulado)

# Convertir a DataFrame para análisis
df_sims = pd.DataFrame(np.array(resultados_acumulados).T, index=df_proj_impuestos.index)

Ejecutando 5000 simulaciones a 12 meses...


  0%|          | 0/5000 [00:00<?, ?it/s]

In [26]:
# ----------------------------------------------------
# ANÁLISIS DE RIESGO
# ----------------------------------------------------
# Probabilidad de Fallo de Liquidez: % de simulaciones donde el Flujo Acumulado cae por debajo del Umbral Mínimo
probabilidad_fallo = (df_sims.min(axis=0) < UMBRAL_LIQUIDEZ_MINIMA).sum() / num_sims
saldo_final_esperado = df_sims.iloc[-1, :].mean()
saldo_final_percentil_10 = np.percentile(df_sims.iloc[-1, :], 10) # Escenario pesimista (10% peor)


In [27]:
# ----------------------------------------------------
# VISUALIZACIÓN SUGERIDA: Tarjeta de KPI
# ----------------------------------------------------
print("\n### Tarjeta de KPI de Riesgo")

kpi_risk_data = [
    {"Indicador": "Probabilidad de Fallo de Liquidez", "Valor": f"{probabilidad_fallo:.2%}", "Propósito": "Riesgo de necesitar financiación externa"},
    {"Indicador": "Saldo Final Esperado (Media)", "Valor": f"{saldo_final_esperado:,.0f} COP", "Propósito": "Escenario más probable"},
    {"Indicador": "Saldo Final Pesimista (P10)", "Valor": f"{saldo_final_percentil_10:,.0f} COP", "Propósito": "Límite inferior de riesgo (10% peor)"}
]
df_kpi_risk = pd.DataFrame(kpi_risk_data)

fig_kpi_risk_table = go.Figure(data=[go.Table(
    header=dict(values=list(df_kpi_risk.columns), fill_color='paleturquoise', align='left'),
    cells=dict(values=[df_kpi_risk['Indicador'], df_kpi_risk['Valor'], df_kpi_risk['Propósito']], fill_color='lavender', align='left'))
])

fig_kpi_risk_table.update_layout(title_text='Indicadores de Riesgo Financiero Clave (KPIs)')
fig_kpi_risk_table.update_layout(
    width=600,
    height=400,
    margin=dict(l=40, r=40, t=60, b=40),
    template="plotly_white"
)
fig_kpi_risk_table.show()



### Tarjeta de KPI de Riesgo


In [28]:
# ----------------------------------------------------
# VISUALIZACIÓN SUGERIDA: Gráfico de Líneas Múltiples (Simulaciones)
# ----------------------------------------------------
print("\n### Gráfico de Líneas Múltiples: Escenarios de Flujo de Caja Acumulado")
fig_sims = go.Figure()

# Trazar 50 líneas de ejemplo (Escenario de Banda)
for col in df_sims.columns[:50]:
    fig_sims.add_trace(go.Scatter(x=df_sims.index, y=df_sims[col], mode='lines',
                                  line=dict(color='rgba(0, 0, 255, 0.1)'), showlegend=False))

# Línea del Escenario Promedio
fig_sims.add_trace(go.Scatter(x=df_sims.index, y=df_sims.mean(axis=1), mode='lines',
                              line=dict(color='blue', width=2), name='Media Proyectada'))

# Banda Roja de Umbral
fig_sims.add_hline(y=UMBRAL_LIQUIDEZ_MINIMA, line_dash="dash", line_color="red",
                   annotation_text=f"Umbral de Liquidez Mínima",
                   annotation_position="top right")

fig_sims.update_layout(title=f'Simulación de Monte Carlo: Flujo de Caja Acumulado ({num_sims} escenarios)',
                        xaxis_title='Fecha de Proyección',
                        yaxis_title='Flujo Neto Acumulado (COP)')
fig_sims.update_layout(
    width=900,
    height=450,
    margin=dict(l=40, r=40, t=60, b=40),
    template="plotly_white",
    font=dict(size=12)
)
fig_sims.show()


# [Image of a risk matrix showing severity vs probability]



### Gráfico de Líneas Múltiples: Escenarios de Flujo de Caja Acumulado


In [29]:
# ----------------------------------------------------
# VISUALIZACIÓN SUGERIDA: Histograma de Resultados Finales
# ----------------------------------------------------
print("\n### Histograma de Resultados: Distribución de Saldos Finales de Caja")

fig_hist = px.histogram(x=df_sims.iloc[-1, :],
                        title='Distribución de Posibles Saldos Finales de Caja',
                        labels={'x': 'Saldo Final de Caja (COP)', 'y': 'Frecuencia'})

fig_hist.add_vline(x=UMBRAL_LIQUIDEZ_MINIMA, line_dash="dash", line_color="red",
                   annotation_text=f"Umbral de Liquidez Mínima", annotation_position="top right")
fig_hist.update_layout(
    width=900,
    height=450,
    margin=dict(l=40, r=40, t=60, b=40),
    template="plotly_white",
    font=dict(size=12)
)
fig_hist.show()


### Histograma de Resultados: Distribución de Saldos Finales de Caja
